In [95]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [25]:
a = torch.arange(6).reshape(2, 3)
print(a)

b = a +1
print(b)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[1, 2, 3],
        [4, 5, 6]])


In [26]:
a0 = torch.tensor([1, 2, 3, 4])
a1 = torch.arange(8).reshape(2, 4)
print(a0)
print(a1)

tensor([1, 2, 3, 4])
tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])


In [30]:
torch.matmul(a1, a0)

tensor([20, 60])

In [33]:
# 行列の積（バッチ）

a0 = torch.arange(24).reshape(-1, 2, 4)
a1 = torch.arange(24).reshape(-1, 4, 2)

torch.matmul(a0, a1)

tensor([[[  28,   34],
         [  76,   98]],

        [[ 428,  466],
         [ 604,  658]],

        [[1340, 1410],
         [1644, 1730]]])

In [36]:
print(a0.dtype)
print(a0.type())


torch.int64
torch.LongTensor


In [56]:
a0 = torch.arange(24).reshape(-1, 2, 4).float()
print('a0の型：',a0.dtype)

c = a0.long()
print('cの型：', c.dtype)
print(f'c形成後のa0の型：{a0.dtype}') # 変化なし　castingはimmutable

a0の型： torch.float32
cの型： torch.int64
c形成後のa0の型：torch.float32


In [63]:
# torchとnumpyへの変換

a0 = torch.arange(3)
print(a0.dtype)

b0 = a0.numpy()
print(b0.dtype)

a1 = torch.from_numpy(b0)
print(a1.dtype)

torch.int64
int64
torch.int64


In [68]:
# 微分をもったtonsorのnumpy変換は不可 -> detch()で切り離す必要がある

a = torch.tensor([1.], requires_grad=True)
print(a)
print(a.dtype)

a.numpy()

tensor([1.], requires_grad=True)
torch.float32


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [70]:
a.detach().numpy()

array([1.], dtype=float32)

In [71]:
np.array([1.]).dtype

dtype('float64')

In [74]:
# tensorの結合
a = torch.zeros(6).reshape(2, 3)
b = torch.ones(6).reshape(2, 3)

# 基本はvstack
print(torch.cat((a, b)))

# hstack
print(torch.cat((a,b), dim=1))


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.]])


In [79]:
# 2次元の配列のリストをバッチ化
a = torch.zeros(6).reshape(2, 3)
b = torch.ones(6).reshape(2, 3)
c = a + 2

torch.stack([a, b, c])

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.]],

        [[2., 2., 2.],
         [2., 2., 2.]]])

In [81]:
# 軸の追加

a = torch.zeros(6).reshape(2, 3)
b = a.unsqueeze(0)
print(a.shape)
print(b.shape)
print(b)

torch.Size([2, 3])
torch.Size([1, 2, 3])
tensor([[[0., 0., 0.],
         [0., 0., 0.]]])


In [85]:
# 軸の入れ替え
a = torch.arange(12).reshape(2, 2, 3)
print(a)

torch.permute(a, (2, 0, 1))


tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])


tensor([[[ 0,  3],
         [ 6,  9]],

        [[ 1,  4],
         [ 7, 10]],

        [[ 2,  5],
         [ 8, 11]]])

## 自動微分

In [87]:
x1 = torch.tensor([1.], requires_grad=True)
x2 = torch.tensor([2.], requires_grad=True)
x3 = torch.tensor([3.], requires_grad=True)

z = (x1 - 2*x2 -1)**2 + (x2*x3 - 1)**2 + 1

z.backward()
x1.grad

tensor([-8.])

In [94]:
def f(x):
    return (x[0]-2*x[1]-1)**2 + (x[1]*x[2]-1)**2 + 1


def f_grad(x):
    z = f(x)
    z.backward()
    return x.grad


x = torch.tensor([1., 2., 3., ], requires_grad=True)

for i in range(50):
    x = x - 0.1 * f_grad(x)
    x = x.detach().requires_grad_(True)

print(f"x = {x.data}, f = {f(x).item()}")

x = tensor([-0.2087, -0.6063, -1.6405]), f = 1.0000436305999756


## Pytorchの学習プログラムの作成

___
・基本はdatasetでデータ準備（呼び出し＋加工）、dataloaderでそのデータを呼び出す。  
・NLPではDataLoaderを使うのは面倒なのであまり使用しない？？

In [98]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# データ準備
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.5)

# 型の指定
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# モデルの定義